# Recap
- we have seen the main idea and concepts from object-oriented programming: classes with attributes and methods. Objects as instances of classes
- we have created custom classes by using the `__init__` method to define attributes

Now we are ready for Transformers and Estimators

# Definition

**Transformers** are special type of classes in python that can be used to make transformations on your data. In the introduction notebook we have seen a diagram that describes a series of operations on your data:

<img src='../images/diagram_data_transformer.png'>

The red part can be defined in python as a series of transformers that take data of a certain type (matrix, dataframe, json) as input and output *transformed* data of the same type (matrix, dataframe, json).

- **TRANSFORMER 1** takes data and extract numerical features from it
- **TRANSFORMER 2** replace missing values with a defined method
- **TRANSFORMER 3** get data from all the transformations sequences in the diagram and concatenate the result into 

Usually transformers have two methods:
- **fit** this method does output anything but it "learns" information from the data
- **transform**: this method applied the transformation to the input and returns the transformed output

Some transformations don't need to learn anything from the data (ex: adding a constant column to a dataframe). These transformations can be performed by **stateless** transformers, that means transformer where the `fit` method does not do anything. Transformers that learn something from the data via the `fit` method (ex: a transformer that add a new column based on other columns values) are called **statefull**

**Estimators** are python classes that "estimate", "learn" from data via a `fit` method. An estimator might be a regression, classification algorithm or even a (statefull) transformer.

<font>

# Examples of scikit-learn Transformers/Estimators

One-Hot Encoder (Transformer)
    https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
    
Logistic Regression (Estimator)
    https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
    
Normalizer (stateless Transformer)
    https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html

# Custom Transformer and Estimators

In scikit-learn it is possible to write your own transformers and estimators to perform transformations on data or to use a custom model for making predictions.
In scikit-learn custom transformers are written as classes that inherit attributes and methods from two special scikit-learn classes: ``BaseEstimator`` and ``TransformerMixin``

```python
from sklearn.base import BaseEstimator, TransformerMixin

class CustomTransformer(BaseEstimator, TransformerMixin):
    ....
```

```BaseEstimator``` https://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html
This is a python class that represents the base from all estimators. An estimator can inherit methods from the BaseEstimator like ```get_param```

```TransformerMixin``` https://scikit-learn.org/stable/modules/generated/sklearn.base.TransformerMixin.html
A Mixin is a class that contains method used by other classes without having to be parent class of those other classes (ex. the method ``fit_transform`` that concatenates fit+transform methods)

### Examples

In [1]:
#Example of a Transformer that does not do anything
from sklearn.base import BaseEstimator, TransformerMixin

class LazyTransformer(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, x, y = None):
        return self

    def transform(self, x):
        return x


In [2]:
class AddTransformer(BaseEstimator,TransformerMixin):
    
    def __init__(self,n):
        self.n = n
        
    def fit(self,x, y=None):
        return self
    
    def transform(self,x):
        return x + self.n

In [3]:
a = AddTransformer(n=3)

In [4]:
x = 1
a.fit(x)

AddTransformer(n=3)

In [6]:
a.transform(x)

4

Exercises
----------

1. Write a transformer that adds some number to the input, the number that is added should be passed in `__init__`. Is this transformer statefull or stateless?
2. Write a transformer that normalizes using the zero-mean normalization method (*hit*: the fit method must 'learn' the mean and the std deviation from the data). Is this transformer statefull or stateless?
3. Combine these 2 transformers into a pipeline:
   - hint: write a class that accepts list of transformers as argument

# Solutions

**Exercise 1**

<div>

class AdderTransformer(TransformerMixin):
    
    def __init__(self, add=0):
        self.add = add
        
    def fit(self, x, y = None):
        return self
    
    def transform(self, x):
        return x + self.add
</div>

**Exercise 2**

<div class='spoiler'>

class ZeroMeanNormalizer(TransformerMixin):
    
    def __init__(self, add=0):
        self.add = add
        
    def fit(self, x, y = None):
        self.mean = x.mean(axis=0)
        self.std_dev = x.std(axis=0)
        return self
    
    def transform(self, x):
        if self.std_dev==0:
            return x-self.mean
        else:
            return (x - self.mean)/self.std_dev    
</div>

**Exercise 3**

<div>
    
class TransformerPipeline(TransformerMixin):
    
    def __init__(self, transformers):
        self.transformers = transformers
        
    def fit(self, x, y = None):
        x_ = x.copy()
        for transformer in self.transformers:
            x_ = transformer.fit_transform(x_)
        return self
        
    def transform(self, x):
        x_ = x.copy()
        for transformer in self.transformers:
            x_ = transformer.transform(x_)
        return x_
</div>

In [16]:
class AdderTransformer(TransformerMixin):
    
    def __init__(self, add=0):
        self.add = add
        
    def fit(self, x, y = None):
        return self
    
    def transform(self, x):
        return x + self.add

class ZeroMeanNormalizer(TransformerMixin):
    
    def __init__(self, add=0):
        self.add = add
        
    def fit(self, x, y = None):
        self.mean = x.mean(axis=0)
        self.std_dev = x.std(axis=0)
        return self
    
    def transform(self, x):
        if self.std_dev==0:
            return x-self.mean
        else:
            return (x - self.mean)/self.std_dev    


In [14]:
class TransformerPipeline(TransformerMixin):
    
    def __init__(self, transformers):
        self.transformers = transformers
        
    def fit(self, x, y = None):
        x_ = x.copy()
        for transformer in self.transformers:
            x_ = transformer.fit_transform(x_)
        return self
        
    def transform(self, x):
        x_ = x.copy()
        for transformer in self.transformers:
            x_ = transformer.transform(x_)
        return x_

In [18]:
a = AdderTransformer(3)
b = ZeroMeanNormalizer()
tp = TransformerPipeline([a,b])

In [19]:
tp.fit(x=np.array([1,2,1,3]))

In [21]:
tp.transform(x=np.array([1,2,1,3]))

array([-0.90453403,  0.30151134, -0.90453403,  1.50755672])